<a href="https://colab.research.google.com/github/YounSooKimTech/Joy_Automation/blob/main/PreDocWebsiteScrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [209]:
# ! pip install beautifulsoup4 requests

# RA Opportunities page

In [ ]:
# Due to the characteristics in the page
# I used text extraction with index for the information
# maybe, using text[index:] can be a better way to extract information I guess

In [210]:
from bs4 import BeautifulSoup
import requests

url = "https://predoc.org/ra-opportunities"
response = requests.get(url, verify=False)
soup = BeautifulSoup(response.text, 'html.parser')

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'predoc.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [211]:
div_tags = soup.find_all("div", class_ = "body-copy")

institutions = []
fields = []
deadlines = []
researchers =[]

for div_tag in div_tags:
  li_tags = div_tag.find_all("li")
  for li_tag in li_tags:
    temp_text = li_tag.get_text()
    inst_index = temp_text.find("ion")
    field_index = temp_text.find("Field")
    dead_index = temp_text.find("Deadline")
    job_index = temp_text.find("Job")

    inst = temp_text[inst_index+4:field_index]
    institutions.append(inst.strip())

    field = temp_text[field_index:dead_index].split(":")[1].strip()
    fields.append(field)

    deadline = temp_text[dead_index:job_index].split(":")[1].strip().replace("Location", "").strip()
    deadlines.append(deadline)

    researcher_index = temp_text.find("Researcher")
    if researcher_index:
      temp_researcher = temp_text[researcher_index:inst_index-8]
      temp = temp_researcher.split(":")[-1]
      researchers.append(temp.strip())

In [212]:
links = []

for div_tag in div_tags:
  a_tags = div_tag.find_all("a")
  for a_tag in a_tags:
    link = a_tag["href"]
    links.append(link)

links = links[4:]

In [213]:
import pandas as pd

df_ra = pd.DataFrame({"institution":institutions,
                      "researchers":researchers,
                      "fields": fields,
                      "deadlines":deadlines,
                      "hyperlinks":links})

df_ra = df_ra[["deadlines", "hyperlinks", "institution", "researchers", "fields"]]


# Opportunities

In [215]:
# All the values can be single or multiples
# The text data are from div (class = "copy")
# I assumed that the information posted in opportunities are aligned in the same format
# I got the hyperlinks from div.copy a.general-link

# the code for the link needs to be corrected

In [214]:
from bs4 import BeautifulSoup
import requests

url = "https://predoc.org/opportunities"
response = requests.get(url, verify=False)
soup = BeautifulSoup(response.text, 'html.parser')


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'predoc.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [216]:
# Deadlines
# nested under div > p > strong > Deadline

copy_divs = soup.find_all('div', class_='copy')

deadlines = []

for copy_div in copy_divs:
    p_tags = copy_div.find_all("p")
    for p_tag in p_tags:
        strong_tags = p_tag.find_all('strong')
        for strong_tag in strong_tags:
          # print(strong_tag.get_text(), strong_tag.next_sibling, "\n")
          if "Deadline" in strong_tag.get_text():
            # print(strong_tag.next_sibling)
            next_sibling = strong_tag.next_sibling
            if next_sibling:
              deadlines.append(next_sibling.strip())
            else:
              deadlines.append("")  # Add an empty string if "researchers" is not found

print(len(deadlines))

58


In [217]:
# Institutions
# nested under div > p > strong (Sponsoring Institution or )>em

copy_divs = soup.find_all('div', class_='copy')

institutions = []

for copy_div in copy_divs:
    p_tags = copy_div.find_all("p")
    for p_tag in p_tags:
        strong_tags = p_tag.find_all('strong')
        for strong_tag in strong_tags:
          if "Institution" in strong_tag.get_text():
            em_tag = strong_tag.find_next("em")
            institutions.append(em_tag.get_text())
            #print(em_tag.get_text())

len(institutions)

58

In [218]:
# Fields
# nested under div > p > strong> Fields of Research

copy_divs = soup.find_all('div', class_='copy')

fields = []

for copy_div in copy_divs:
    p_tags = copy_div.find_all("p")
    for p_tag in p_tags:
        strong_tags = p_tag.find_all('strong')
        for strong_tag in strong_tags:
          # print(strong_tag.get_text(), strong_tag.next_sibling, "\n")
          if "Field" in strong_tag.get_text():
            # print(strong_tag.next_sibling)
            next_sibling = strong_tag.next_sibling
            if next_sibling:
              fields.append(next_sibling.strip().strip(":"))
            else:
              fields.append("")  # Add an empty string if "researchers" is not found


print(len(fields))

58


In [219]:
# Researchers

copy_divs = soup.find_all('div', class_='copy')
researchers = []

for copy_div in copy_divs:
    p_tags = copy_div.find_all("p")
    for p_tag in p_tags:
      #print(p_tag.get_text(), "\n")
      if "Researcher" in p_tag.get_text() or "Researcher(s)" in p_tag.get_text():
        strong_tags = p_tag.find_all('strong')
        for strong_tag in strong_tags:
          if "Researcher" in strong_tag.get_text() or "Researcher(s)" in strong_tag.get_text() :
              next_sibling = strong_tag.next_sibling
              researchers.append(next_sibling.strip().strip(":").strip("()"))
      else:
        researchers.append("None")

len(researchers)

58

In [220]:
# hyperlinks in the div> a >href

links = []

a_tags = soup.select('div.copy a.general-link')

for a_tag in a_tags:
    # print(a_tag["href"])
    links.append(a_tag["href"])
len(set(links))

120

In [221]:
# make the dataframe

import pandas as pd

df = pd.DataFrame({"institution":institutions,
                   "researchers":researchers,
                  "fields": fields,
                   "deadlines":deadlines,
                   "hyperlinks":links[0:len(institutions)]
                   })

df.sample(5)

,institution,researchers,fields,deadlines,hyperlinks
46,Harvard Business School,: Prof. Olivia Kim in the Entrepreneurial Mana...,"Finance, microeconomics, management",: Rolling,https://bit.ly/45fxH5W
20,"Becker Friedman Institute for Economics (BFI),...",John List,Experimental economics,"October 15, 2023",https://bit.ly/3sWLC2Q
6,MIT Sloan School of Management,"Tong Liu, Lira Mota, Christopher Palmer, and K...","Capital markets, corporate finance, macro-fina...",Apply ASAP to receive full consideration,https://bit.ly/3LETDjg
9,The Becker Friedman Institute for Economics at...,Magne Mogstad and Evan Rose,"Applied econometrics, labor, and public economics","October 15, 2023",https://bit.ly/3PnF9p4
11,The University of Chicago Booth School of Busi...,Richard Hornbeck and Anders Humlum,"Microeconomics, Economic History","The priority deadline is October 16, 2023. Sub...",https://bit.ly/48lXXh9


# merging the whole dataset

In [226]:
df1 = df

import pandas as pd

df = pd.concat([df_ra, df1], axis=0)
df

,deadlines,hyperlinks,institution,researchers,fields,deadline
0,"October 13, 2023",https://bit.ly/3LFuCod,DOJ Pathways Internship Program Bureau of Prisons,,Psychology,NaN
1,"October 13, 2023",https://bit.ly/3PXATOv,"Department of Commerce, U.S. Census Bureau",,Interpretation of geographical and statistical...,NaN
2,"October 10, 2023",https://bit.ly/46fL6eX,The Federal Energy Regulatory Commission (FERC),,"Economics, environmental, energy industry, ele...",NaN
3,"October 16, 2023",https://bit.ly/3PY7DHB,African Development Foundation,,Finance and budget analyst functions,NaN
4,"October 16, 2023",https://bit.ly/3LmEvqJ,The U.S. Government Accountability Office (GAO),,Financial audits and analyses,NaN
...,...,...,...,...,...,...
2,Accepting applications on a rolling basis.,https://bit.ly/3tbB9Rh,Stanford University,Matthew Gentzkow,Applied microeconomics,rolling
1,Accepting applications on a rolling basis.,https://bit.ly/3rGpitM,"Behavior Change for Good Initiative, The Whart...",Katy Milkman and Angela Duckworth,Behavioral Science,rolling
41,Rolling,https://bit.ly/3YEoy4D,"Princeton University, Julis-Rabinowitz Center ...",Atif Mian and Ernest Liu,"Empirical macrofinance, macroeconomics, financ...",rolling
28,Rolling,https://bit.ly/461m1o9,Kellogg School of Management at Northwestern U...,Profs. Yuval Salant and Jörg Spenkuch,Managerial Economics and Decision Sciences,rolling


# handling the date file

In [227]:
# extract the date from the deadline text

import re
from datetime import datetime

date_pattern = r"(?i)((?:January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{1,2})"

df['deadline'] = df['deadlines'].str.extract(date_pattern, expand=False)
df.loc[df['deadline'].isna() & df['deadlines'].str.contains(r'(?i)rolling'), 'deadline'] = 'rolling'
df.loc[df['deadline'].isna() & df['deadlines'].str.contains(r'(?i)ASAP'), 'deadline'] = 'ASAP'

df = df.sort_values(by = "deadline", ascending=True)

In [228]:
# get the opportunities whose deadline is latter than today

df1 = df[(df["deadline"] != "ASAP") & (df["deadline"] != "rolling")]

df1['deadline'] = pd.to_datetime(df1['deadline'] + " 2023", format='%B %d %Y', errors='coerce')

today = datetime.now().date()

subset_df = df1[df1['deadline'].dt.date >= today]


# make the format of deadline as yyyy-mm-dd foramt
subset_df['deadline'] = subset_df['deadline'].dt.strftime('%Y-%m-%d')

# sort the subset
subset_df = subset_df.sort_values(by="deadline", ascending=True)

subset_df = subset_df[["deadline", "hyperlinks", "institution", "researchers", "fields"]]

<ipython-input-228-a87b19bad59a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['deadline'] = pd.to_datetime(df1['deadline'] + " 2023", format='%B %d %Y', errors='coerce')
<ipython-input-228-a87b19bad59a>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['deadline'] = subset_df['deadline'].dt.strftime('%Y-%m-%d')


In [229]:
subset_df.head()

,deadline,hyperlinks,institution,researchers,fields
12,2023-10-04,https://bit.ly/3PO68M4,The University of Chicago Booth School of Busi...,,"Microeconomics, public economics, inequality, ..."
24,2023-10-06,https://bit.ly/3r9X22G,Opportunity Insights,"Raj Chetty (Harvard), John Friedman (Harvard/B...","Economics, Public Economics, Public Policy, Bi..."
21,2023-10-06,https://bit.ly/3Rtwa8i,"University of Notre Dame, Wilson Sheehan Lab f...",Bill Evans,"Applied Microeconomics, Economics of Education..."
2,2023-10-10,https://bit.ly/46fL6eX,The Federal Energy Regulatory Commission (FERC),,"Economics, environmental, energy industry, ele..."
0,2023-10-13,https://bit.ly/3LFuCod,DOJ Pathways Internship Program Bureau of Prisons,,Psychology


In [235]:
df2 = df[(df["deadline"] == "ASAP") | (df["deadline"] == "rolling")]
df2 = df2[["deadline", "hyperlinks", "institution", "researchers", "fields"]]

df2

,deadline,hyperlinks,institution,researchers,fields
43,ASAP,https://bit.ly/47lzEzk,IESE Business School,Professor Tammaro Terracciano,"Empirical research on fintech, banking, intern..."
6,ASAP,https://bit.ly/3LETDjg,MIT Sloan School of Management,"Tong Liu, Lira Mota, Christopher Palmer, and K...","Capital markets, corporate finance, macro-fina..."
7,ASAP,https://bit.ly/3rxh3jQ,MIT Sloan School of Management,"Larry Schmidt, Taha Choukhmane, Christopher Pa...","Asset pricing, household and consumer finance,..."
26,rolling,https://bit.ly/3T27VMI,University of Pennsylvania (The Wharton School),"Arthur van Benthem, Susanna Berkouwer and Shin...",Energy and Environmental/Development Economics
23,rolling,https://bit.ly/45Se6ct,"NBER & the University of Chicago, Center for t...","Director Professor James Heckman, and/or Execu...",Economics of early childhood development
5,rolling,https://bit.ly/46bAtcY,"Princeton University, Industrial Relations Sec...",Zach Bleemer,"Labor Economics, Applied Econometrics and Stat..."
4,rolling,https://bit.ly/3rBOx0k,"Stanford University, Regulation, Evaluation, &...",Daniel E. Ho,"Computer Science, Data Science, Statistics, Ma..."
2,rolling,https://bit.ly/3tbB9Rh,Stanford University,Matthew Gentzkow,Applied microeconomics
1,rolling,https://bit.ly/3rGpitM,"Behavior Change for Good Initiative, The Whart...",Katy Milkman and Angela Duckworth,Behavioral Science
30,rolling,https://bit.ly/3LdjHBK,"The University of California, Los Angeles","Dan Benjamin, David Cesarini, Patrick Turley, ...",Social-science genomics


In [230]:
# download the processed file in excel format

excel_file_name = 'predoc_information_with_deadline.xlsx'
subset_df.to_excel(excel_file_name, index=False)

from google.colab import files
files.download(excel_file_name)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>